<a href="https://colab.research.google.com/github/seungwonlee003/Item-Recyclability-Identification/blob/main/Recyclability_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from PIL import Image

model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the pre-trained weights in the base model
for layer in model.layers:
    layer.trainable = False
    

# Add new layers on top of the pre-trained ResNet50 model
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

# Load the ResNet50 model
model = tf.keras.applications.ResNet50(weights='imagenet')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.5, 1.5])

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'file path for training dataset',
    target_size=(224, 224),
    batch_size=12,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    'file path for validation dataset',
    target_size=(224, 224),
    batch_size=12,
    class_mode='categorical')

model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the pre-trained weights in the base model
for layer in model.layers[:-10]:
    layer.trainable = False

# Add new layers on top of the pre-trained ResNet50 model
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(6, activation='softmax')(x)

# Build the final model
model = tf.keras.Model(inputs=model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=50, validation_data=valid_generator)

# Evaluate the model
model.evaluate(valid_generator)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the class labels
class_labels = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

# Load an image
img = load_img('file path for actual image', target_size=(224, 224))

# Preprocess the image
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make a prediction
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class label
predicted_class_label = class_labels[predicted_class_index]

# Determine if the predicted class is recyclable or not
if predicted_class_label == "trash":
    print("The item is not recyclable.")
else:
    print("The item is recyclable.")